* 方法参考自 The Barra China Equity Model (CNE5)'s 文档

* 请在环境变量中设置`DB_URI`指向数据库

In [ ]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import statsmodels.api as sm
from alphamind.api import *
from PyFin.api import *
from alphamind.analysis.crosssetctions import cross_section_analysis

plt.style.use('ggplot')

In [ ]:
"""
Back test parameter settings
"""

start_date = '2016-01-01'
end_date = '2018-02-28'
category = 'sw'
level = 1
freq = '20b'
universe = Universe('ashare_ex')

horizon = map_freq(freq)
ref_dates = makeSchedule(start_date, end_date, freq, 'china.sse')

In [ ]:
def risk_factor_analysis(factor_name):
    data_source = os.environ['DB_URI']
    engine = SqlEngine(data_source)
    risk_names = list(set(risk_styles).difference({factor_name}))
    industry_names = list(set(industry_styles).difference({factor_name}))
    constraint_risk = risk_names + industry_names
    
    df = pd.DataFrame(columns=['ret', 'ic', 't.'], dtype=float)

    for ref_date in ref_dates:
        df.loc[ref_date, :] = cross_section_analysis(ref_date,
                                                     factor_name,
                                                     universe,
                                                     horizon,
                                                     constraint_risk,
                                                     engine=engine)
    df.index = pd.to_datetime(df.index)
    return df

In [ ]:
candidates_factors = ['BETA', 'SIZE']

In [ ]:
%%time
res = [risk_factor_analysis(factor) for factor in candidates_factors]

In [ ]:
df = pd.DataFrame()

for f_name, data in zip(candidates_factors, res):
    data['factor'] = f_name
    df = df.append(data)

In [ ]:
df['abs t.'] = np.abs(df['t.'])
df[['factor', 'abs t.']].groupby('factor').mean().sort_values('abs t.', ascending=False).head()